In [1]:
import numpy as np
import pandas as pd
import scipy.spatial as sc
import sqlite3 as sql
import plotly.express as px

import heliolinc3d as hl
import heliolinc3d.heliolinc3d_v3 as hl2

import logging
import os

from numba import njit
from sklearn.cluster import DBSCAN
from time import time

from heliolinc3d.transforms import radec2icrfu, frameChange
from heliolinc3d.vector import sphere_line_intercept
from heliolinc3d import constants as cn

from numpy import float32

In [2]:
logger = logging.getLogger()
logger.setLevel( logging.INFO )
logging.debug(' initilizing logger...')

## HLCpy? HLinCpy? LinCpy? (generalize to any frame?)

### HelioLinC python demo

For this demo we look at only observations corresponding to actual (simulated) objects, and do not include false positives from source processing and sensor noise (which is ~10x the number of points and causes memory issues).

This demo instead looks at what clustering parameter is needed to recover good clusters with dbscan

### Load Detections

we load a 2-week window of observations 

In [3]:
# load dataset
dfObs = pd.read_parquet( '../neo_mba_noise_v3.3_1month.parquet', filters=[('ObjID', '!=', 'FD'), ('FieldMJD_TAI', '<=', 60810.0)])
# dfObs = pd.read_parquet( '../neo_mba_noise_v3.3_1month.parquet', filters=[('FieldMJD_TAI', '<=', 60810.0)])
dfObs.rename( 
    columns = {
        'FieldMJD_TAI' : 'FieldMJD',
        'index' : 'obsName' 
    }, 
    inplace=True )
dfObs

,ObjID,FieldID,FieldMJD,AstRA(deg),AstDec(deg),observedTrailedSourceMag,optFilter
0,G003061,24,60796.01243,210.812507,-73.626046,22.920,r
1,G003061,74,60796.03689,210.781540,-73.625285,23.479,g
2,G004759,201,60796.09643,169.145905,-7.835319,22.064,u
3,G004759,251,60796.11954,169.149066,-7.826511,20.579,g
4,G002012,252,60796.11999,168.356483,-10.641907,24.105,g
...,...,...,...,...,...,...,...
6202912,S101MR5wa,13251,60809.15669,230.762016,10.495020,22.982,r
6202913,S101N7kka,13227,60809.14595,193.019583,9.064945,23.724,r
6202914,S101NDxUa,13121,60809.09520,187.341383,7.238919,23.842,r
6202915,S101Nt2Sa,13132,60809.10013,180.363544,-5.266361,24.226,r


In [4]:

mjd_start = dfObs[ 'FieldMJD'].min()
mjd_stop = mjd_start + 14.0
mjd_start, mjd_stop

# obs_batch.drop( obs_batch['FieldMJD'] > mjd_stop, inplace=True )
# # dfObs = dfObs[ dfObs['FieldMJD_TAI'] <= mjd_stop ]
# mjd_ref = 0.5*(mjd_start + mjd_stop)

(60796.00164, 60810.00164)

In [5]:
# image_db = sql.connect( '../opsims/baseline_v3.3_10yrs.db' )
# image_table = pd.read_sql_query( 'SELECT * FROM observations', image_db )
# image_table

# obs_pos, obs_vel = hl.getObserverStates(
#     image_table['observationStartMJD'],
#     origin='Sun',
#     observer_location='X05',
#     ephemeris_dt='1h',
#     frame='ecliptic'
#     )

# obs_states = pd.DataFrame( {
#     'image_id' : image_table['observationId'],
#     'epoch_mjd' : image_table['observationStartMJD'],
#     'x_obs' : obs_pos[:,0],
#     'y_obs' : obs_pos[:,1],
#     'z_obs' : obs_pos[:,2],
#     'vx_obs' : obs_vel[:,0],
#     'vy_obs' : obs_vel[:,1],
#     'vz_obs' : obs_vel[:,2],
# } )
# obs_states.to_csv( 'demo/X05_states_v3.3_10yrs.db', index=False )

In [6]:
obs_states = pd.read_csv( 'demo/X05_states_v3.3_10yrs.csv', )
obs_states

,image_id,epoch_mjd,x_obs,y_obs,z_obs,vx_obs,vy_obs,vz_obs
0,0,60796.001439,-0.765787,-0.654735,0.000013,0.010787,-0.013323,0.000080
1,1,60796.001888,-0.765782,-0.654741,0.000013,0.010787,-0.013323,0.000080
2,2,60796.002335,-0.765777,-0.654747,0.000013,0.010788,-0.013324,0.000080
3,3,60796.002782,-0.765772,-0.654753,0.000013,0.010789,-0.013324,0.000080
4,4,60796.003233,-0.765768,-0.654759,0.000013,0.010789,-0.013324,0.000080
...,...,...,...,...,...,...,...,...
2130903,2130903,64448.427310,-0.767134,-0.653015,0.000051,0.011066,-0.013061,-0.000047
2130904,2130904,64448.427781,-0.767129,-0.653022,0.000051,0.011065,-0.013060,-0.000048
2130905,2130905,64448.428240,-0.767124,-0.653028,0.000051,0.011065,-0.013060,-0.000048
2130906,2130906,64448.428692,-0.767119,-0.653033,0.000051,0.011065,-0.013059,-0.000048


In [7]:
dfObs = pd.merge( dfObs, obs_states, how='left', left_on='FieldID', right_on='image_id',)

In [8]:
# set parameters
r0 = 1.2
r0dot = 0.001
clust_rad = 0.000001
max_ang_rate = 2*np.pi/180 # deg/day => rad/day 
dt_trk = 2/24 # days
mjd_ref = 0.5*(mjd_stop + mjd_start) # we use the obs start from the image table
mjd_ref

60803.00164

In [9]:
# get line of sight vectors
dfObs['x_los'], dfObs['y_los'], dfObs['z_los'] = radec2icrfu( 
    dfObs['AstRA(deg)'],
    dfObs['AstDec(deg)']
 )

In [10]:
# testing numbers of tracklets 
xyzt = np.zeros( (len(dfObs), 4) )
xyzt[:, 0:3] = sphere_line_intercept( 
    dfObs[['x_los', 'y_los', 'z_los']].values.T,
    dfObs[['x_obs', 'y_obs', 'z_obs']].values.T,
    1.2
 ).T
xyzt[:, 3] = dfObs['epoch_mjd'].values * 0.02

In [11]:
trk_tree = sc.KDTree( xyzt, leafsize=16 )

In [12]:
n_pairs = trk_tree.count_neighbors( trk_tree, r=0.02*2.0/24 )

In [13]:
(n_pairs - len( dfObs )) / 2, (n_pairs - len( dfObs )) / 2 / len(dfObs)

(466659596.0, 75.23228119931316)

In [14]:
#  75 pairs per obs (   466659596 (4.67 e8) total ) at 1.2 au with 0.02 au / day velocity search, no false_positives
# 200 pairs per obs ( 15408951551 (15.4 e9) total ) at 1.2 au with 0.02 au / day velocity search, with false_positives ( 10 min )

In [15]:
# x_arrows, v_arrows, dt_arrows, pairs = 
pairs = hl2.find_tracklets_on_sky(
    dfObs[['x_los', 'y_los', 'z_los']].values.T,
    dfObs[['x_obs', 'y_obs', 'z_obs']].values.T,
    dfObs['epoch_mjd'].values
    )

INFO:heliolinc3d.heliolinc3d_v3: building tracklet search tree ...
INFO:heliolinc3d.heliolinc3d_v3: tracklet search tree construction time 2.3054280281066895 s
INFO:heliolinc3d.heliolinc3d_v3: querying for neighbors with search radius 0.002908882086657216
INFO:heliolinc3d.heliolinc3d_v3: query time 9.231968879699707 s
INFO:heliolinc3d.heliolinc3d_v3: removing same time pairs


In [19]:
pairs.shape, pairs.shape[0] / 466659596

((128789672, 2), 0.27598205009374754)

In [23]:
128789672 * 2*8 * 1e-9, 466659596 * 2*8 * 1e-9

(2.0606347520000003, 7.466553536)

In [ ]:
arrows = hl2.make_arrows_from_pairs( 
    pairs, 
    dfObs[['x_los', 'y_los', 'z_los']].values, 
    dfObs[['x_obs', 'y_obs', 'z_obs']].values, 
    dfObs['epoch_mjd'].values-mjd_ref, 
    r0, 
    r0dot, 
    n_iter=3, GM=cn.GM )

In [ ]:
arrows

In [ ]:
arrows.shape

In [ ]:
arrows.shape[0] * 6 * 8 * 1e-9 # GB

In [ ]:
velocity_mask = np.where( ~np.any( np.isnan(arrows), axis=1 ) )
pairs = pairs[ velocity_mask ]
arrows = arrows[ velocity_mask ]

In [ ]:
arrows.shape, pairs.shape

In [ ]:
arrows.shape[0] * 6 * 8 * 1e-9

In [ ]:
x_arrows

In [ ]:
L_arrows = np.cross( x_arrows, v_arrows, axis=0 )
L_arrows.T

In [ ]:
A_arrows = np.cross( v_arrows, L_arrows, axis=0 ) - cn.GM*x_arrows/np.linalg.norm(x_arrows, axis=0)
A_arrows.T / cn.GM
# e = np.linalg.norm( A_arrows / cn.GM, axis=0 ) 

In [ ]:
e = np.linalg.norm( A_arrows / cn.GM, axis=0 ) 
e.max(), e.min()

In [ ]:
# scale A to L
A_arrows /= np.linalg.norm( A_arrows, axis=0 )
A_arrows *= np.linalg.norm( L_arrows, axis=0 )

In [ ]:
points = np.vstack([L_arrows, A_arrows]).T
clusters = DBSCAN( eps=0.00002, min_samples=3 ).fit( points )
len( np.unique(clusters.labels_) )

In [ ]:
tracklets = pd.DataFrame({
    'observation_1'   : pairs[0],
    'observation_1_id': dfObs.iloc[ pairs[0] ]['ObjID'].values,
    'mjd_1'           : dfObs.iloc[ pairs[0] ]['FieldMJD'].values,
    'img_1'           : dfObs.iloc[ pairs[0] ]['FieldID'].values,
    'observation_2'   : pairs[1],
    'observation_2_id': dfObs.iloc[ pairs[1] ]['ObjID'].values,
    'mjd_2'           : dfObs.iloc[ pairs[1] ]['FieldMJD'].values,
    'img_2'           : dfObs.iloc[ pairs[1] ]['FieldID'].values,
    'l_x'             : L_arrows[0],
    'l_y'             : L_arrows[1],
    'l_z'             : L_arrows[2],
    'e_x'             : A_arrows[0]/cn.GM,
    'e_y'             : A_arrows[1]/cn.GM,
    'e_z'             : A_arrows[2]/cn.GM,
    'res2_l'           : 0.0,
    'res2_e'           : 0.0,
    'cluster'         : clusters.labels_,
})

In [ ]:
# dfObs[ dfObs['ObjID'] == 'S1002pjea' ]

In [ ]:
# len(dfObs[ dfObs['ObjID'] == 'S1002pjea' ]['FieldID'].unique())

In [ ]:
# tracklets[ tracklets['cluster']==11 ]

In [ ]:
clusters_dedup = []
for cluster_label, cluster in tracklets.groupby( 'cluster' ):
    imgs1 = np.unique( cluster['img_1'] )
    imgs2 = np.unique( cluster['img_2'] )
    names = np.array([ np.unique(cluster['observation_1_id']), np.unique(cluster['observation_1_id']) ]).flatten()

    for name in names:
        if name[0] == 'G':
            print( '\n', cluster_label )
            print( (cluster[['observation_1', 'img_1', 'observation_1_id',]] ))
            print( (cluster[['observation_2', 'img_2', 'observation_2_id',]] ))
            break


    # if len( imgs1 ) == len(cluster) and len( imgs2 ) == len(cluster):
    #     clusters_dedup.append( cluster )
    #     print( '\n', cluster_label )
    #     print( (cluster[['observation_1', 'img_1', 'observation_1_id',]] ))
    #     print( (cluster[['observation_2', 'img_2', 'observation_2_id',]] ))
        # print( (cluster[['observation_1', 'img_1', 'observation_1_id',]] ))
    # else: # split cluster on duplicate observations
    #     print( '\n', cluster_label )
    #     print( (cluster[['observation_1', 'img_1', 'observation_1_id',]] ))
    #     print( (cluster[['observation_2', 'img_2', 'observation_2_id',]] ))
        # for _, trk in cluster.iterrows():

# clusters = pd.concat( clusters_dedup )

In [ ]:
# compute mean, residuals
cluster_mean_states = []
for cluster_label, cluster in tracklets.groupby( 'cluster' ):
    # print( cluster['l_x'] 
    l_mean = np.mean( cluster[['l_x', 'l_y', 'l_z']], axis=0 )
    e_mean = np.mean( cluster[['e_x', 'e_y', 'e_z']], axis=0 )

    l_std = np.std( cluster[['l_x', 'l_y', 'l_z']], axis=0 )
    e_std = np.std( cluster[['e_x', 'e_y', 'e_z']], axis=0 )
    # l_std_norm = np.linalg.norm(l_std)

    # res2_l = np.sum( ( cluster[['l_x', 'l_y', 'l_z']] - l_mean )**2, axis=0 )
    # res2_e = np.sum( ( cluster[['e_x', 'e_y', 'e_z']] - e_mean )**2, axis=0 )
    cluster_mean_states.append( np.hstack([cluster_label, l_mean, e_mean, l_std, e_std]) )
    # cluster
    # n = len( np.unique( cluster[['observation_1_id', 'observation_2_id']].values.flatten() ) )
    # if l_std_norm < 3*clust_rad:
    #     print( l_std, l_std_norm, n )
cluster_mean_states = np.vstack( cluster_mean_states )
cluster_mean_states = pd.DataFrame(cluster_mean_states, columns = [
    'cluster', 
    'mean_l_x', 'mean_l_y', 'mean_l_z', 
    'mean_e_x', 'mean_e_y', 'mean_e_z',
    'std_l_x', 'std_l_y', 'std_l_z', 
    'std_e_x', 'std_e_y', 'std_e_z',
    ])

In [ ]:
cluster_mean_states['std_l'] = np.sqrt( np.sum(cluster_mean_states[['std_l_x', 'std_l_y', 'std_l_z']].values**2, axis=1) )
cluster_mean_states['std_e'] = np.sqrt( np.sum(cluster_mean_states[['std_e_x', 'std_e_y', 'std_e_z']].values**2, axis=1) )
cluster_mean_states.iloc[1:]

In [ ]:
# visualize standard deviations of the clusters ex ey
px.density_heatmap( cluster_mean_states.iloc[1:], x='std_l', y='std_e', nbinsx=100, nbinsy=100 )
# px.histogram( cluster_mean_states, x='std_l', y='std_e' )

In [ ]:
tracklets

In [ ]:
tracklets = pd.merge( tracklets, cluster_mean_states, on='cluster' )
tracklets[['res_l_x', 'res_l_y', 'res_l_z', 'res_e_x', 'res_e_y', 'res_e_z']] = tracklets[['l_x', 'l_y', 'l_z', 'e_x', 'e_y', 'e_z']].values - tracklets[['mean_l_x', 'mean_l_y', 'mean_l_z', 'mean_e_x', 'mean_e_y', 'mean_e_z']].values
# tracklets

In [ ]:
tracklets

In [ ]:
pairs_idx = np.array([[i,i] for i in range(pairs.shape[1])]).T.flatten()
# pairs_idx = np.array([[i,i] for i in range(pairs.shape[1])]).T.flatten()
obs_idx = pairs.flatten()
cluster_obs = pd.DataFrame({
    'obs_id' : obs_idx,
    'trk_idx' : pairs_idx,
    'cluster': clusters.labels_[ pairs_idx ],
    'obj_id' : dfObs.iloc[obs_idx]['ObjID'].values,
    'obs_mjd' : dfObs.iloc[obs_idx]['FieldMJD'].values,
})
cluster_obs = cluster_obs[ cluster_obs['cluster'] != -1 ]
cluster_obs

In [ ]:
# deduplicate
#
for cluster_ird, group in cluster_obs.groupby( 'cluster' ):
    print(group)

In [ ]:
# groups = cluster_obs.groupby( 'obs_id' )
# group = groups.get_group( 2924789 )
# tracklets.iloc[ group['trk_idx'] ]
# # groups.keys

In [ ]:
# filtered_trcklets = []
# groups = cluster_obs.groupby( 'obs_id' )
# for obs, group in cluster_obs.groupby( 'obs_id' ) :
#     # trks_in_group = tracklets.iloc[ group['trk_idx'] ]
#     if len(group) > 1:
#         print( obs )

#     # best_trk = 

#     # print(trks_in_group)

In [ ]:
tracklets = pd.DataFrame({
    'observation_1'   : pairs[0],
    'observation_1_id': dfObs.iloc[ pairs[0] ]['ObjID'].values,
    'mjd_1'           : dfObs.iloc[ pairs[0] ]['FieldMJD'].values,
    'observation_2'   : pairs[1],
    'observation_2_id': dfObs.iloc[ pairs[1] ]['ObjID'].values,
    'mjd_2'           : dfObs.iloc[ pairs[1] ]['FieldMJD'].values,
    'l_x'             : L_arrows[0],
    'l_y'             : L_arrows[1],
    'l_z'             : L_arrows[2],
    'e_x'             : A_arrows[0]/cn.GM,
    'e_y'             : A_arrows[1]/cn.GM,
    'e_z'             : A_arrows[2]/cn.GM,
    'cluster'         : clusters.labels_
})
# tracklets = tracklets[ tracklets['cluster'] != -1 ]
tracklets

In [ ]:
# # remove tracklets which share observations if they result in seperate clusters
# # leaving the one with the best cluster match
# filtered_tracklets = []
# for obs_1, group in tracklets.groupby( 'observation_1_id' ):
#     # identify best tracklet

In [ ]:
for cluster_label, cluster in tracklets.groupby( 'cluster' ):
    # print( cluster['l_x'] 
    l_mean = np.mean( cluster[['l_x', 'l_y', 'l_z']], axis=0 )
    l_std = np.std( cluster[['l_x', 'l_y', 'l_z']], axis=0 )
    l_std_norm = np.linalg.norm(l_std)
    n = len( np.unique( cluster[['observation_1_id', 'observation_2_id']].values.flatten() ) )
    if l_std_norm < 3*clust_rad:
        print( l_std, l_std_norm, n )